In [259]:
!pip install transformers==4.9.2
!pip install nltk
!pip install datasets==1.14.0
!pip install sentencepiece
!pip install torch==1.8.0+cu111 torchvision==0.9.0+cu111 torchaudio==0.8.0 -f https://download.pytorch.org/whl/torch_stable.html

  Using cached huggingface_hub-0.0.12-py3-none-any.whl (37 kB)
ERROR: datasets 1.14.0 has requirement huggingface-hub<0.1.0,>=0.0.19, but you'll have huggingface-hub 0.0.12 which is incompatible.
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.0.19
    Uninstalling huggingface-hub-0.0.19:
      Successfully uninstalled huggingface-hub-0.0.19
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Using cached huggingface_hub-0.0.19-py3-none-any.whl (56 kB)
ERROR: transformers 4.9.2 has requirement huggingface-hub==0.0.12, but you'll have huggingface-hub 0.0.19 which is incompatible.
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.0.12
    Uninstalling huggingface-hub-0.0.12:
      Successfully uninstalled huggingface-hub-0.0.12
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location

# play

In [260]:
import sys
sys.path.append('/UnifiedSKGG-CYPHER')

In [261]:
import os
import time
import torch
import datasets
from transformers import (
    HfArgumentParser,
    set_seed,
    AutoTokenizer
)
from utils.configue import Configure
from utils.training_arguments import WrappedSeq2SeqTrainingArguments

from filelock import FileLock
import nltk
with FileLock(".lock") as lock:
    nltk.download("punkt", quiet=True)
    nltk.download("stopwords", quiet=True)

INFO:filelock:Lock 140003947357232 acquired on .lock
INFO:filelock:Lock 140003947357232 released on .lock


In [262]:
def play(txt, model, tokenizer):
  print("=====❓Request=====")
  print(txt)
  tokenized_txt = tokenizer([txt], max_length=1024, padding="max_length", truncation=True)
  pred = tokenizer.batch_decode(
      model.generate(
        torch.LongTensor(tokenized_txt.data['input_ids']),
        torch.LongTensor(tokenized_txt.data['attention_mask']),
        num_beams=1, 
        max_length=256
        ), 
      skip_special_tokens=True 
  ) # More details see utils/dataset.py and utils/trainer.py
  print("=====💡Answer=====")
  print(pred)
  return pred

In [263]:
#  Spider (with cell value)

# Set args here for runnning on notebook, we make them out here to make it more illustrative.
sys.argv = ['/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py', # This is the name of your .py launcher when you run this line of code.
            # belows are the parameters we set, take spider for example
            '--cfg', 'Salesforce/T5_base_prefix_spider_with_cell_value.cfg', 
            '--output_dir', './tmp']
parser = HfArgumentParser((WrappedSeq2SeqTrainingArguments,))
training_args, = parser.parse_args_into_dataclasses()
set_seed(training_args.seed)
args = Configure.Get(training_args.cfg)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [264]:
# Load tokenizer and model(21->1 multitasked prefix)
tokenizer = AutoTokenizer.from_pretrained("hkunlp/from_all_T5_base_prefix_spider_with_cell_value2", use_fast=False)
from models.unified.prefixtuning import Model
model = Model(args)
model.load("hkunlp/from_all_T5_base_prefix_spider_with_cell_value2")

loading file https://huggingface.co/hkunlp/from_all_T5_base_prefix_spider_with_cell_value2/resolve/main/spiece.model from cache at /home/22842219/.cache/huggingface/transformers/c137efa00ca821483894a38692ad1c725ea36d72bbd5b659538cdd041695674c.d6f0605ae3d57070be74b4c12206072ab332922acff822e6b5458691dbda7551
loading file https://huggingface.co/hkunlp/from_all_T5_base_prefix_spider_with_cell_value2/resolve/main/added_tokens.json from cache at /home/22842219/.cache/huggingface/transformers/416d14f1fa4c64761400b6f1fd0ce39bcc8a2e0583e22219dc154f3c5c74b7aa.ffb6426fb410c59e57e5a4c85d9a0c58498b73237acd2662871aa605056dcc5a
loading file https://huggingface.co/hkunlp/from_all_T5_base_prefix_spider_with_cell_value2/resolve/main/special_tokens_map.json from cache at /home/22842219/.cache/huggingface/transformers/15f4ad6c9cbcaa98da6ffb8009488d50a7d655f5135340fd7eef0d58d914c8a9.c94798918c92ded6aeef2d2f0e666d2cc4145eca1aa6e1336fde07f2e13e2f46
loading file https://huggingface.co/hkunlp/from_all_T5_base_

prefix-tuning sequence length is 10.


Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/t5-base/resolve/main/config.json from cache at /home/22842219/.cache/huggingface/transformers/91e9fe874e06c44883b535d6c950b8b89d6eaa3298d8e7fb3b2c78039e9f8b7b.a085189636e38420df5e7bdf08ad1b86f1fe33c010079ca7f15437ff95f4fe2b
Model config T5Config {
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "gradient_checkpointing": false,
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": t

In [265]:
struct_in = "| concert_singer | stadium : stadium_id , location , name , capacity , highest , lowest , average | singer : singer_id , name , country ( France ) , song_name , song_release_year , age , is_male | concert : concert_id , concert_name , theme , stadium_id , year | singer_in_concert : concert_id , singer_id"
text_in = "what is the minimum, average, and maximum age of all singers from France?"
# seq_out = "SELECT avg(age) ,  min(age) ,  max(age) FROM singer WHERE country  =  'France'"

In [266]:
generated_sql = play("{}; structed knowledge: {}".format(text_in, struct_in), model, tokenizer)

=====❓Request=====
what is the minimum, average, and maximum age of all singers from France?; structed knowledge: | concert_singer | stadium : stadium_id , location , name , capacity , highest , lowest , average | singer : singer_id , name , country ( France ) , song_name , song_release_year , age , is_male | concert : concert_id , concert_name , theme , stadium_id , year | singer_in_concert : concert_id , singer_id
=====💡Answer=====
['select min(age), avg(age), max(age) from singer where country = "France"']


In [267]:
tokenized_txt = tokenizer([text_in], max_length=1024, padding="max_length", truncation=True)
print("tokenized_txt:", tokenized_txt)
print("tokenized_txt:", len(tokenized_txt['input_ids'][0]))
print("tokenized_txt:", len(tokenized_txt['attention_mask'][0]))

tokenized_txt: {'input_ids': [[125, 19, 8, 2559, 6, 1348, 6, 11, 2411, 1246, 13, 66, 7634, 7, 45, 1410, 58, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [268]:
generated_sql

['select min(age), avg(age), max(age) from singer where country = "France"']

In [269]:
'''
Author: Ziyu Zhao
Affiliation: UWA NLT-TLP GROUP
'''

import os, re, math
from typing import Set

import sqlite3
import pandas as pd

class DBengine:
    
    """
    An Entity, which takes a sqlite3 database schema.
    Attributes:
    
    """

    def __init__(self, fdb):

        self.conn = sqlite3.connect(fdb)      
        self.conn.text_factory = lambda b: b.decode(errors = 'ignore')

    
    def execute(self, query):
      cursor = self.conn.cursor()  
      return cursor.execute(query)  
    
    def get_table_names(self):
        cursor = self.conn.cursor()  
        return cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")     
    
    def get_table_values(self, table_name):
        cursor = self.conn.cursor()  
        return cursor.execute('SELECT * from {}'.format(table_name))  

    def get_schema(self, table_name):
        cursor = self.conn.cursor()  
        cursor.execute('SELECT sql FROM sqlite_master WHERE type = "table" and name= "{}";'.format(table_name))  
        result = cursor.fetchone()
        if result is None:
            raise ValueError("Table %s does not exist" % table_name)
        return result[0].strip()  

    def get_outbound_foreign_keys(self, table_name):
        cursor = self.conn.cursor() 
        infos = cursor.execute("PRAGMA foreign_key_list([{}])".format(table_name)).fetchall()
        table_constraints = []
        pks_fks_dict ={}
        for info in infos:
            if info is not None:
                id, seq, ref_table, from_, to_, on_update, on_delete, match = info
                table_constraints.append(
                {"this_table": table_name, "column": from_, "ref_table": ref_table, "ref_column": to_}
                )
                pks_fks_dict[from_]= to_  # ref_column (aka pk in ref_table) is supposed to be the same with column. 
        return table_constraints, pks_fks_dict

    def get_primay_keys(self, table_name):
        cursor = self.conn.cursor() 
        pks = cursor.execute('SELECT l.name FROM pragma_table_info("{}") as l WHERE l.pk <> 0;'.format(table_name)).fetchall()
        return pks  
    
    def check_compound_pk(self, primary_keys):
        compound_pk_check=False
        if len(primary_keys)>1: 
            compound_pk_check=True
        return compound_pk_check

    def close(self):
        self.conn.cursor.close()

In [270]:
db_path = '/home/22842219/Desktop/openSource/UnifiedSKGG-Cypher/data/downloads/spider/database/concert_singer/concert_singer.sqlite'
engine = DBengine(db_path)
for sql_query in generated_sql:
    print(sql_query)
    sql_ans = engine.execute(sql_query).fetchall()
    print("sql_ans:", sql_ans)

select min(age), avg(age), max(age) from singer where country = "France"
sql_ans: [(25, 34.5, 43)]


In [271]:
!pip install py2neo

In [272]:

from py2neo import Graph
import configparser

config = configparser.ConfigParser()
config.read('/home/22842219/Desktop/phd/SemanticParser4Graph/application/config.ini')
filenames = config["FILENAMES"]

neo4j_uri = filenames['neo4j_uri']
neo4j_user = filenames['neo4j_user']
neo4j_password = filenames['neo4j_password']
graph = Graph(neo4j_uri, auth = (neo4j_user, neo4j_password))

In [273]:
sql2cypher = "MATCH (singer:`concert_singer.singer`) \
WHERE singer.Country = 'France' \
RETURN min(singer.Age),avg(singer.Age),max(singer.Age)"
cypher_res = graph.run(sql2cypher).data()
print(cypher_res)

[{'min(singer.Age)': 25, 'avg(singer.Age)': 34.5, 'max(singer.Age)': 43}]


# end-to-end translation from text-to-cypher

In [1]:
########################## unifiedSKG learning process*******************
import logging
import os
import time

import torch
import datasets
import transformers
from transformers import (
    HfArgumentParser,
    set_seed,
    EarlyStoppingCallback,
)
from transformers.trainer_utils import get_last_checkpoint
from utils.configue import Configure
from utils.training_arguments import WrappedSeq2SeqTrainingArguments

from collections import OrderedDict
import utils.tool
from utils.dataset import TokenizedDataset
from utils.trainer import EvaluateFriendlySeq2SeqTrainer

# Huggingface realized the "Seq2seqTrainingArguments" which is the same with "WrappedSeq2SeqTrainingArguments"
# in transformers==4.10.1 during our work.
logger = logging.getLogger(__name__)



/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [ ]:
args = Configure.Get(training_args.cfg)
print(training_args.cfg)
print(args.bert, args.bert.location)

In [325]:
print(training_args)

WrappedSeq2SeqTrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
cfg=Salesforce/T5_base_prefix_spider_with_cell_value.cfg,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
generation_max_length=512,
generation_num_beams=4,
gradient_accumulation_steps=1,
greater_is_better=None,
group_by_length=False,
ignore_data_skip=False,
input_max_length=1536,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=5e-05,
length_column_name=length,
load_best_model_at_end=False,
load_weights_from=None,
local_rank=-1,
log_level=-1,
log_level_replica=-1,
log_on_each_node=True,
logging_dir=./tmp/runs/Jan29_14-57-29_DEP24073.uniwa.uwa.edu

In [326]:
if 'checkpoint-???' in args.bert.location:
    args.bert.location = get_last_checkpoint(
        os.path.dirname(args.bert.location.model_name_or_path))
    logger.info(f"Resolve model_name_or_path to {args.bert.location.model_name_or_path}")


In [327]:
last_checkpoint = None
if os.path.isdir(
        training_args.output_dir) and training_args.do_train and not training_args.overwrite_output_dir:
    last_checkpoint = get_last_checkpoint(training_args.output_dir)
    if last_checkpoint is None and len(os.listdir(training_args.output_dir)) > 0:
        raise ValueError(
            f"Output directory ({training_args.output_dir}) already exists and is not empty. "
            "Use --overwrite_output_dir to overcome."
        )
    elif last_checkpoint is not None and training_args.resume_from_checkpoint is None:
        logger.info(
            f"Checkpoint detected, resuming training at {last_checkpoint}. To avoid this behavior, change "
            "the `--output_dir` or add `--overwrite_output_dir` to train from scratch."
        )

os.makedirs(training_args.output_dir, exist_ok=True)

In [288]:
print(not args.arg_paths)
print(args.dataset.loader_path)  ## hey, attention!
print(args.dataset.data_store_path)
print(args.seq2seq.constructor)
print(args.arg_paths)

False
None
./data/text2cypher
seq2seq_construction.meta_tuning


In [328]:
# The inputs will be train, dev, test or train, dev now.
# We deprecate the k-fold cross-valid function 
# since it causes too many avoidable troubles.
if not args.arg_paths:
    cache_root = os.path.join('output', 'cache')
    os.makedirs(cache_root, exist_ok=True)
    raw_datasets_split: datasets.DatasetDict = datasets.load_dataset(path=args.dataset.loader_path,
                                                                        cache_dir=args.dataset.data_store_path)
    seq2seq_dataset_split: tuple = utils.tool.get_constructor(args.seq2seq.constructor)(args).to_seq2seq(
        raw_datasets_split, cache_root)


In [329]:
cache_root = os.path.join('output', 'cache')
print(f'cache_root: {cache_root}')
os.makedirs(cache_root, exist_ok=True)
meta_tuning_data = {}


cache_root: output/cache


In [330]:
for task, arg_path in args.arg_paths:
    task_args = Configure.Get(arg_path)
    task_args.bert = args.bert
    print(f'task: {task}, arg_path: {arg_path}, task_args: {task_args}')
    print('task_args.bert.location:', task_args.bert.location)
    print(f'task_args.dataset: {task_args.dataset}, task_args.dataset.loader_path: {task_args.dataset.loader_path}')
    print(f'task_args.dataset.data_store_path: {task_args.dataset.data_store_path}')
    task_raw_datasets_split: datasets.DatasetDict = datasets.load_dataset(
        path=task_args.dataset.loader_path,
        cache_dir=task_args.dataset.data_store_path)
    print(f'task_raw_datasets_split: {task_raw_datasets_split}')

    print(f'task_args.seq2seq.constructor: {task_args.seq2seq.constructor}')
    task_seq2seq_dataset_split: tuple = utils.tool.get_constructor(task_args.seq2seq.constructor)(task_args).\
        to_seq2seq(task_raw_datasets_split, cache_root)

    meta_tuning_data[arg_path] = task_seq2seq_dataset_split
    print(f'arg_path: {arg_path}')
    print(f'meta_tuning_data: {meta_tuning_data}')

seq2seq_dataset_split: tuple = utils.tool.get_constructor(args.seq2seq.constructor)(args).\
    to_seq2seq(meta_tuning_data)
print(f'seq2seq_dataset_split: {seq2seq_dataset_split}')

task: spider, arg_path: META_TUNING/spider_with_cell.cfg, task_args: <utils.configue.Args object at 0x7f55280826a0>
task_args.bert.location: t5-base
task_args.dataset: <utils.configue.Args object at 0x7f553846f6a0>, task_args.dataset.loader_path: ./tasks/spider.py
task_args.dataset.data_store_path: ./data


100%|██████████| 2/2 [00:00<00:00, 402.47it/s]

task_raw_datasets_split: DatasetDict({
    train: Dataset({
        features: ['query', 'question', 'db_id', 'db_path', 'db_table_names', 'db_column_names', 'db_column_types'],
        num_rows: 4730
    })
    validation: Dataset({
        features: ['query', 'question', 'db_id', 'db_path', 'db_table_names', 'db_column_names', 'db_column_types'],
        num_rows: 705
    })
})
task_args.seq2seq.constructor: seq2seq_construction.spider


arg_path: META_TUNING/spider_with_cell.cfg
meta_tuning_data: {'META_TUNING/spider_with_cell.cfg': (<seq2seq_construction.spider.TrainDataset object at 0x7f55334d0160>, <seq2seq_construction.spider.DevDataset object at 0x7f553846f160>)}
seq2seq_dataset_split: (<seq2seq_construction.meta_tuning.TrainDataset object at 0x7f5520a7ab50>, <seq2seq_construction.meta_tuning.DevDataset object at 0x7f5520a7a190>, <seq2seq_construction.meta_tuning.TestDataset object at 0x7f5502bdd550>)


In [333]:
print(task_args)
print(args)


In [332]:
print(f'args: {args}, args.evaluate.tool: {args.evaluate.tool}')
evaluator = utils.tool.get_evaluator(args.evaluate.tool)(args)
print(f'evaluator: {evaluator}')

args: <utils.configue.Args object at 0x7f552012fc70>, args.evaluate.tool: metrics.meta_tuning.evaluator
evaluator: <metrics.meta_tuning.evaluator.EvaluateTool object at 0x7f550c7a8820>


In [334]:
print(f'args.model.name: {args.model.name}, args: {args}')
model = utils.tool.get_model(args.model.name)(args)

args.model.name: unified.prefixtuning, args: <utils.configue.Args object at 0x7f552012fc70>
prefix-tuning sequence length is 10.


Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/t5-base/resolve/main/config.json from cache at /home/22842219/.cache/huggingface/transformers/91e9fe874e06c44883b535d6c950b8b89d6eaa3298d8e7fb3b2c78039e9f8b7b.a085189636e38420df5e7bdf08ad1b86f1fe33c010079ca7f15437ff95f4fe2b
Model config T5Config {
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "gradient_checkpointing": false,
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": t

In [338]:
model_tokenizer = model.tokenizer
model_tokenizer

PreTrainedTokenizer(name_or_path='t5-base', vocab_size=32100, model_max_len=512, is_fast=False, padding_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>', '<extra_id_43>', '<extra_id_44>', '<extra_id_45>',

In [336]:
seq2seq_train_dataset, seq2seq_eval_dataset, seq2seq_test_dataset = None, None, None
print(f'seq2seq_train_dataset: {seq2seq_train_dataset}')

seq2seq_train_dataset: None


In [337]:
print(f'len(seq2seq_dataset_split): {len(seq2seq_dataset_split)}')
if len(seq2seq_dataset_split) == 2:
    seq2seq_train_dataset, seq2seq_eval_dataset = seq2seq_dataset_split
elif len(seq2seq_dataset_split) == 3:
    seq2seq_train_dataset, seq2seq_eval_dataset, seq2seq_test_dataset = seq2seq_dataset_split
    print(f'seq2seq_train_dataset: {seq2seq_train_dataset}, seq2seq_eval_dataset: {seq2seq_eval_dataset} \
        seq2seq_test_dataset: {seq2seq_test_dataset}')
else:
    raise ValueError("Other split not support yet.")

len(seq2seq_dataset_split): 3
seq2seq_train_dataset: <seq2seq_construction.meta_tuning.TrainDataset object at 0x7f5520a7ab50>, seq2seq_eval_dataset: <seq2seq_construction.meta_tuning.DevDataset object at 0x7f5520a7a190>         seq2seq_test_dataset: <seq2seq_construction.meta_tuning.TestDataset object at 0x7f5502bdd550>


In [339]:
from utils.dataset import TokenizedDataset

In [340]:
# We wrap the "string" seq2seq data into "tokenized tensor".
train_dataset = TokenizedDataset(args, training_args, model_tokenizer,
                                    seq2seq_train_dataset) if seq2seq_train_dataset else None
eval_dataset = TokenizedDataset(args, training_args, model_tokenizer,
                                seq2seq_eval_dataset) if seq2seq_eval_dataset else None
test_dataset = TokenizedDataset(args, training_args, model_tokenizer,
                                seq2seq_test_dataset) if seq2seq_test_dataset else None
print(f'train_dataset: {train_dataset}')
print(f'eval_dataset: {eval_dataset}')
print(f'test_dataset: {test_dataset}')


train_dataset: <utils.dataset.TokenizedDataset object at 0x7f55229c5340>
eval_dataset: <utils.dataset.TokenizedDataset object at 0x7f55229c52e0>
test_dataset: <utils.dataset.TokenizedDataset object at 0x7f55229c53a0>


In [341]:
# Initialize our Trainer
early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=args.seq2seq.patience if args.seq2seq.patience else 5)
print(f'early_stopping_callback: {early_stopping_callback}')
print(f'patience: {args.seq2seq.patience}')

early_stopping_callback: <transformers.trainer_callback.EarlyStoppingCallback object at 0x7f550c7a8d30>
patience: 200


In [342]:
from utils.trainer import EvaluateFriendlySeq2SeqTrainer


In [343]:
print(f'seq2seq_eval_dataset: {seq2seq_eval_dataset}')

seq2seq_eval_dataset: <seq2seq_construction.meta_tuning.DevDataset object at 0x7f5520a7a190>


In [303]:
!export WANDB_API_KEY=edd277e1d782e96634109705a6235577032da246
!export WANDB_PROJECT='text-to-sql(text2cypher-24Jan)'
!export WANDB_ENTITY=leamonzea929

In [353]:
print(training_args.report_to)
print(training_args.local_rank <=0)
print("MLFLOW_EXPERIMENT_ID" in os.environ)
print(os.getenv("WANDB_PROJECT", "uni-frame-for-knowledge-tabular-tasks"))
print(training_args.run_name)
print(os.getenv("WANDB_ENTITY", 'sgtnew'))

['wandb']
True
False
uni-frame-for-knowledge-tabular-tasks
./tmp
sgtnew


In [354]:
trainer = EvaluateFriendlySeq2SeqTrainer(
    args=training_args,
    model=model,
    evaluator=evaluator,
    # We name it "evaluator" while the hugging face call it "Metric",
    # they are all f(predictions: List, references: List of dict) = eval_result: dict
    tokenizer=model_tokenizer,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    eval_examples=seq2seq_eval_dataset,
    # wandb_run_dir=wandb.run.dir if "wandb" in training_args.report_to and training_args.local_rank <= 0 else None,
    callbacks=[early_stopping_callback],
)
print('Trainer build successfully.')

Trainer build successfully.


In [345]:
print(training_args.load_weights_from)

# Load model weights (for --do_train=False or post finetuning).
if training_args.load_weights_from:
    state_dict = torch.load(os.path.join(training_args.load_weights_from, transformers.WEIGHTS_NAME), map_location="cpu")
    trainer.model.load_state_dict(state_dict, strict=True)
    # release memory
    del state_dict

None


In [348]:
print(args.load_multiple_prefix_module_weights_from)
if args.load_multiple_prefix_module_weights_from:
    reconstruct_state_dict = OrderedDict()

    # load prefix modules
    for task_name, module_weight_location in args.load_multiple_prefix_module_weights_from:
        state_dict = torch.load(os.path.join(module_weight_location, transformers.WEIGHTS_NAME), map_location="cpu")
        MULTI_PREFIX_ATTR_NAME = "multi_prefix"
        for weight_name, stored_tensor in state_dict.items():
            if str(weight_name).startswith("pretrain_model"):
                continue  # skip the pretrained model and we will load a new one from another place
            reconstruct_state_dict['{}.{}.{}'.format(MULTI_PREFIX_ATTR_NAME, "_".join(task_name.split("_")[:-1]), weight_name)] = stored_tensor
            # extract the prefix part and add them to dict

    # give it into the model
    trainer.model.load_state_dict(reconstruct_state_dict, strict=False)

    # release memory
    del reconstruct_state_dict

None
